In [1]:
pwd

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/trailmet

/content/drive/MyDrive/trailmet


In [4]:
import torch
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory

In [5]:
import yaml

In [6]:
root = '/content/drive/MyDrive/trailmet/experiments/ReActNet'

In [7]:
import os
with open(os.path.join(root,"reactnet_cifar100.yaml"),'r') as stream:
    data_loaded = yaml.safe_load(stream)

In [8]:
data_loaded

{'GENERAL': {'BACKBONE': 'resnet18', 'DATASET': 'CIFAR100'},
 'ReActNet1_ARGS': {'batch_size': 128,
  'epochs': 1,
  'label_smooth': 0.1,
  'learning_rate': 0.0025,
  'momentum': 0.9,
  'weight_deacy': '1e-5'},
 'ReActNet2_ARGS': {'batch_size': 128,
  'epochs': 1,
  'label_smooth': 0.1,
  'learning_rate': 0.0025,
  'momentum': 0.9,
  'weight_deacy': 0}}

In [9]:
model = ModelsFactory.create_model('resnet18', 10, False, insize=32)
teacher = ModelsFactory.create_model('resnet101', 10, False, insize=32)

In [10]:
from trailmet.datasets.classification import DatasetFactory

In [11]:
mkdir data_dir

mkdir: cannot create directory ‘data_dir’: File exists


In [12]:
train_transform = transforms.Compose(
[transforms.ToTensor()])

val_transform = transforms.Compose(
[transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

In [13]:
cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR10', 
                                        root = 'data_dir',
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [14]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train_dataset'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val_dataset'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test_dataset'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )

In [15]:
from trailmet.algorithms.binarize.ReActNet import ReActNet

In [16]:
a = ReActNet(teacher,model, {'train': train_loader, 'val': val_loader, 'test': test_loader}, **data_loaded)

In [17]:
fin = a.compress_model()

Step-1 Training with activations binarized for 1 epochs

EPOCH-0



/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
100%|██████████| 157/157 [00:04<00:00, 35.66it/s, acc=0.0999, loss=2.34]


Average Train Loss-2.535806939315796
Top-1 Train Accuracy-9.695
Top-5 Train Accuracy-50.085
Validation Accuracy-0.0999
Validation Loss-2.3412274372805455
Step-2 Training with both activations and weights binarized for 1 epochs
EPOCH-0



100%|██████████| 157/157 [00:07<00:00, 20.82it/s, acc=0.0948, loss=2.61]

Average Train Loss-3.0345276432037354
Top-1 Train Accuracy-9.795
Top-5 Train Accuracy-50.2775
Validation Accuracy-0.0948
Validation Loss-2.6053057779931716
